In [ ]:
### Fixing import errors of the

import sys
import os

# This code navigates up one directory from the notebook's location ('examples/')
# to get the project's root directory.
project_root = os.path.abspath(os.path.join(os.getcwd(), '..'))

# We check if the path is already in the system path.
# If not, we add it to the beginning of the list.
if project_root not in sys.path:
    sys.path.insert(0, project_root)
    print(f"Added project root to Python path: {project_root}")
else:
    print(f"Project root is already in Python path: {project_root}")

# Optional: You can print the first few paths to verify
print("\nVerifying sys.path:")
for i, path in enumerate(sys.path[:5]):
    print(f"{i}: {path}")

Added project root to Python path: /home/nick/projects/Llama-Index-GliREL-GraphRAG

Verifying sys.path:
0: /home/nick/projects/Llama-Index-GliREL-GraphRAG
1: /usr/lib/python312.zip
2: /usr/lib/python3.12
3: /usr/lib/python3.12/lib-dynload
4: 


In [ ]:
from src.GlirelPathExtractor import GlirelPathExtractor 
from llama_index.core import SimpleDirectoryReader, PropertyGraphIndex
from llama_index.core.graph_stores import SimpleGraphStore
from llama_index.core import Settings
from IPython.display import Markdown, display


/home/nick/envs/grag-llama/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
from llama_index.llms.ollama import Ollama

In [ ]:
llm = Ollama(
    model= "gemma3:12b",
    request_timeout=120.0,
    context_window=8128,
    temperature=0.0
)

Settings.llm = llm
Settings.chunk_size=512
Settings.chunk_overlap=64

In [ ]:
from llama_index.embeddings.ollama import OllamaEmbedding
embed_model = OllamaEmbedding(
    model_name="snowflake-arctic-embed2:latest",
    ollama_additional_kwargs={"mirostat": 0},
)
Settings.embed_model = embed_model

In [ ]:
documents = SimpleDirectoryReader(
    input_files=["../.data/Demo-Barack_Obama.txt"]
).load_data()

In [ ]:
extractor = GlirelPathExtractor(device="cuda")


In [ ]:
index = PropertyGraphIndex.from_documents(
    documents,
    kg_extractors=[extractor],
    use_async = False
    
)


In [ ]:
query_engine = index.as_query_engine(
    llm=llm,
    include_text="True",
    response_mode="tree_summarize",
    embedding_mode="hybrid",
    similarity_top_k=5,
    )

In [ ]:
response= query_engine.query("When was obama born?")
response